Wytłumaczenie kodu:
Dropdown: Pozwala użytkownikowi wybierać lokalizacje.
Mapa: Używam biblioteki dash_leaflet do stworzenia interaktywnej mapy.
Callback: Funkcja w Dash, która aktualizuje pozycję markera i centrum mapy na podstawie wybranej lokalizacji.
Po uruchomieniu tego kodu, Dash utworzy lokalny serwer, na którym będzie dostępna aplikacja z mapą i listą rozwijaną. Użytkownik będzie mógł wybrać lokalizację z listy, a mapa automatycznie skupi się na wybranej lokalizacji.

OPIS tego notebboka zrobic


w dashu1 beda pokazywane ogolne wykresy historyczne i beda dynamicznie sie zminiec na podstawie wybranej lokalizacji, tez obliczone wartosci dla paneli slonecznych

w dashu 2 bedzie to samo tylko z predykcja

Plan działania dla Dashboardu w Dash:

Przygotowanie Środowiska:

Upewnij się, że masz zainstalowane wszystkie niezbędne pakiety, takie jak dash, dash-core-components, dash-html-components, pandas, i plotly.

Wczytanie Danych:

In [5]:
import pandas as pd

# Wczytywanie danych
df_weather_final = pd.read_csv("../2. Data/processed/final_df_processed.csv")


In [6]:
df_weather_final.head(10)

,Unnamed: 0,date,city_name,lat,lon,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,clouds_all,weather_main,estimated_irradiance,temp_modifier,humidity_modifier,daily_energy_production_kWh,year,month
0,0,1979-01-01,Baden_Baden,48.765640,8.228524,-10.442083,-16.532917,-16.18,3.02,1013.666667,81.916667,3.681667,84.500000,Cloudy,2.0,1.0,0.95,7.6,1979,1
1,1,1979-01-01,Bozanska,54.473028,18.302179,-14.100000,-20.617500,-19.14,-9.46,1002.416667,82.000000,6.157917,100.000000,Cloudy,2.0,1.0,0.95,7.6,1979,1
2,2,1979-01-01,Walcz_Drugi,53.257039,16.521203,-16.437083,-23.322500,-21.26,-9.57,1003.708333,82.416667,3.682083,94.791667,Cloudy,2.0,1.0,0.95,7.6,1979,1
3,3,1979-01-02,Baden_Baden,48.765640,8.228524,-8.856667,-14.874167,-15.82,-3.96,1024.250000,84.208333,4.524583,71.708333,Snow,1.5,1.0,0.95,5.7,1979,1
4,4,1979-01-02,Bozanska,54.473028,18.302179,-9.037917,-15.179167,-14.14,-7.40,1004.791667,85.375000,4.449583,99.541667,Cloudy,2.0,1.0,0.95,7.6,1979,1
5,5,1979-01-02,Walcz_Drugi,53.257039,16.521203,-7.259583,-14.093333,-10.04,-5.30,1008.500000,88.458333,5.382500,99.458333,Cloudy,2.0,1.0,0.95,7.6,1979,1
6,6,1979-01-03,Baden_Baden,48.765640,8.228524,-7.245417,-11.859583,-12.99,-2.53,1023.833333,83.333333,2.877917,54.541667,Clouds,2.5,1.0,0.95,9.5,1979,1
7,7,1979-01-03,Bozanska,54.473028,18.302179,-10.780000,-16.679167,-16.33,-6.84,1004.500000,87.041667,3.805833,97.083333,Cloudy,2.0,1.0,0.95,7.6,1979,1
8,8,1979-01-03,Walcz_Drugi,53.257039,16.521203,-8.548750,-15.455417,-11.76,-6.76,1007.625000,87.416667,5.291250,97.791667,Cloudy,2.0,1.0,0.95,7.6,1979,1
9,9,1979-01-04,Baden_Baden,48.765640,8.228524,-8.997083,-15.285417,-13.37,-4.80,1012.291667,81.333333,4.802500,77.125000,Clouds,2.5,1.0,0.95,9.5,1979,1


Budowa Layoutu Dashboardu:
Struktura dashboardu z elementami takimi jak input box do wyboru lokalizacji, wyświetlanie mapy i różnych statystyk.


ja bym chcial miec nastepujcae elementy na swoim dashbordzie:
1. input gdie bedzie mozna wpisywac miejscowosc, mamy trzy tylko wiec jak bedzie wpisywane cos innego to ma zwrocic "nie ma takiej miejscowosci w bazie". Nie chce rozwijanej listy. - DONE

2. wykres sredniej miesiecznej temperatury w danej lokalizacji - DONE 

3.  wykres srdeniego zachmurzenia dla danej lokalizacji - DONE

4. wykres srednia miesieczna wilgotonosci - DONE

5. wykres sredniej miesiecznej preedkosci wiatru dla danej lokalizacji - DONE

6. wykres rocznej produkcji energi przez lata dla danej lokalizacji, zmiana lat zeby byla jako suwak.- DONE

7. wykres sredniej miesiecznej produkcji ennergii dla danej lokalizacji na orzestrzeni lat. - DONE

8. Osobna tabela gdzie bedzie pokazana daily_energy_production_kWh

9. Mapa z lokalizacja - DONE

10. do tabeli jakies koszty?

11. tytul gdziesz ze to tylko dla instalcji przykladowej

12. podac tez gdzies typowe wartsoci dla domow zeby miec porownanie

13. isntniejace wykresy, tytul dodac zmienna NAME dla lkoalizacji.

14. poprawic ten wykres miesieczny w 03_notebook i zrobic ten wykres tutaj, - DONE 

15. zastanowic sie nad uzyciem GEOPANDAS - ?

In [7]:
## STARY KOD do spr i wyrzucenia

# import dash
# from dash import dcc, html, Input, Output, State
# import pandas as pd
# import plotly.express as px
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots

# # Załadowanie danych
# df = pd.read_csv("../2. Data/processed/final_df_processed.csv")
# df['date'] = pd.to_datetime(df['date'])
# df['year'] = df['date'].dt.year
# df['month'] = df['date'].dt.month

# # Dane dla stolic Europy
# capitals_data = {
#     'city_name': ['Berlin', 'Paris', 'Madrid', 'Rome', 'Warsaw', 'London', 'Athens', 'Vienna', 'Budapest', 'Prague', 'Moscow', 'Kiev', 'Oslo', 'Stockholm', 'Helsinki', 'Copenhagen', 'Amsterdam', 'Brussels', 'Lisbon', 'Dublin'],
#     'lat': [52.52, 48.8566, 40.4165, 41.9028, 52.2297, 51.5074, 37.9838, 48.2082, 47.4979, 50.0755, 55.7558, 50.4501, 59.9139, 59.3293, 60.1695, 55.6761, 52.3676, 50.8503, 38.7223, 53.3498],
#     'lon': [13.405, 2.3522, -3.7038, 12.4964, 21.0122, -0.1278, 23.7275, 16.3738, 19.0402, 14.4378, 37.6173, 30.5234, 10.7522, 18.0686, 24.9384, 12.5683, 4.9041, 4.3517, -9.1393, -6.2603]
# }
# capitals_df = pd.DataFrame(capitals_data)

# # Inicjalizacja aplikacji Dash
# app = dash.Dash(__name__)

# # Layout aplikacji
# app.layout = html.Div([
#     dcc.Input(id='city-input', type='text', placeholder='Wpisz nazwę miejscowości'),
#     html.Button('Sprawdź', id='submit-button', n_clicks=0),
#     html.Div("---------------------------------------------"),
#     html.Div("Otwórz w przeglądarce: http://127.0.0.1:8050/"),
#     html.Div("---------------------------------------------"),
#     html.Div("Dostępne miejscowości: Baden_Baden, Bozanska, Walcz_Drugi"),
#     html.Div(id='error-message'),
#     html.Div([
#         dcc.Graph(id='monthly-weather-chart', style={'width': '48%','height': '100vh', 'display': 'inline-block', 'margin-right': '2%'}),
#         dcc.Graph(id='map-chart', style={'width': '48%','height': '100vh', 'display': 'inline-block', 'margin-left': '2%'})
#     ]),
#     html.Div([
#         dcc.Graph(id='annual-energy-chart'),
#         dcc.RangeSlider(
#             id='year-range-slider',
#             min=df['year'].min(),
#             max=df['year'].max(),
#             value=[df['year'].min(), df['year'].max()],
#             marks={str(year): str(year) for year in df['year'].unique()},
#             step=None
#         )
#     ])
# ])

# # Callback dla dynamicznej aktualizacji i mapy
# @app.callback(
#     [Output('error-message', 'children'),
#      Output('city-input', 'style'),
#      Output('annual-energy-chart', 'figure'),
#      Output('monthly-weather-chart', 'figure'),
#      Output('map-chart', 'figure')],
#     [Input('submit-button', 'n_clicks'),
#      Input('year-range-slider', 'value')],
#     [State('city-input', 'value')]
# )
# def update_output(n_clicks, year_range, city):
#     if city not in df['city_name'].unique():
#         return 'Nie ma takiej miejscowości w bazie', {'border': '2px solid red'}, {}, {}, {}

#     filtered_df = df[(df['city_name'] == city) & (df['year'] >= year_range[0]) & (df['year'] <= year_range[1])]
#     annual_energy_fig = px.bar(filtered_df, x='year', y='daily_energy_production_kWh', title=f'Roczna produkcja energii przez lata dla {city}')

#     weather_attributes = ['temp','clouds_all', 'humidity', 'wind_speed']
#     attribute_labels = ['Temperatura (°C)','Zachmurzenie (%)', 'Wilgotność (%)', 'Prędkość wiatru (m/s)']
#     monthly_weather_fig = make_subplots(rows=4, cols=1, subplot_titles=attribute_labels, vertical_spacing=0.2)
#     for i, attr in enumerate(weather_attributes, start=1):
#         monthly_data = filtered_df.groupby('month')[attr].mean().reset_index()
#         monthly_weather_fig.add_trace(go.Scatter(x=monthly_data['month'], y=monthly_data[attr], mode='lines+markers'), row=i, col=1)
#         monthly_weather_fig.update_xaxes(title_text="Miesiące", tickvals=list(range(1, 13)), row=i, col=1)

#     monthly_weather_fig.update_layout(height=800, showlegend=False, title_text="Średnie miesięczne warunki pogodowe na przestrzeni wszystkich lat", title_x=0)
#     monthly_weather_fig.update_yaxes(rangemode='tozero')

#     # Mapa z lokalizacją
#     location_data = df[df['city_name'] == city].drop_duplicates(subset=['city_name'])
#     map_fig = px.scatter_geo(location_data, lat='lat', lon='lon', hover_name='city_name',
#                              projection='natural earth', title='Lokalizacja na mapie Europy',
#                              size_max=20, color_discrete_sequence=["red"])
#     map_fig.add_scattergeo(lat=capitals_df['lat'], lon=capitals_df['lon'], hoverinfo='text',
#                            text=capitals_df['city_name'], marker=dict(size=5, color='blue', symbol='circle'))
#     map_fig.update_geos(
#         visible=True, showcountries=True, countrycolor="Black",
#         showcoastlines=True, coastlinecolor="Black",
#         showland=True, landcolor="LightGreen",
#         projection_scale=2
#     )
#     map_fig.update_layout(
#         geo=dict(
#             scope='europe', projection_type='orthographic', showframe=False,
#             showcoastlines=True, landcolor='rgb(243, 243, 243)',
#             countrycolor='rgb(204, 204, 204)',
#             lataxis_range=[35, 70],  # Ograniczenie zakresu szerokości geograficznej dla Europy
#             lonaxis_range=[-25, 40],  # Ograniczenie zakresu długości geograficznej dla Europy
#             projection_scale=2  # Zwiększ skalę projekcji
#         )
#     )

#     return '', {'border': '1px solid green'}, annual_energy_fig, monthly_weather_fig, map_fig

# if __name__ == '__main__':
#     app.run_server(debug=True)


In [10]:
import dash
from dash import dcc, html, Input, Output, State
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.graph_objects import Bar, Figure

# Załadowanie danych
df = pd.read_csv("../2. Data/processed/final_df_processed.csv")
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

# Dane dla stolic Europy
capitals_data = {
    'city_name': ['Berlin', 'Paris', 'Madrid', 'Rome', 'Warsaw', 'London', 'Athens', 'Vienna', 'Budapest', 'Prague', 'Moscow', 'Kiev', 'Oslo', 'Stockholm', 'Helsinki', 'Copenhagen', 'Amsterdam', 'Brussels', 'Lisbon', 'Dublin'],
    'lat': [52.52, 48.8566, 40.4165, 41.9028, 52.2297, 51.5074, 37.9838, 48.2082, 47.4979, 50.0755, 55.7558, 50.4501, 59.9139, 59.3293, 60.1695, 55.6761, 52.3676, 50.8503, 38.7223, 53.3498],
    'lon': [13.405, 2.3522, -3.7038, 12.4964, 21.0122, -0.1278, 23.7275, 16.3738, 19.0402, 14.4378, 37.6173, 30.5234, 10.7522, 18.0686, 24.9384, 12.5683, 4.9041, 4.3517, -9.1393, -6.2603]
}
capitals_df = pd.DataFrame(capitals_data)

# Inicjalizacja aplikacji Dash
app = dash.Dash(__name__)

# Layout aplikacji
app.layout = html.Div([
    dcc.Input(id='city-input', type='text', placeholder='Wpisz nazwę miejscowości'),
    html.Button('Sprawdź', id='submit-button', n_clicks=0),
    html.Div("---------------------------------------------"),
    html.Div("Otwórz w przeglądarce: http://127.0.0.1:8050/"),
    html.Div("---------------------------------------------"),
    html.Div("Dostępne miejscowości: Baden_Baden, Bozanska, Walcz_Drugi"),
    html.Div(id='error-message'),
    html.Div([
        dcc.Graph(id='monthly-weather-chart', style={'width': '48%','height': '100vh', 'display': 'inline-block', 'margin-right': '2%'}),
        dcc.Graph(id='map-chart', style={'width': '48%','height': '100vh', 'display': 'inline-block', 'margin-left': '2%'})
    ]),
    
    html.Div([
        dcc.Graph(id='monthly-energy-chart', style={'width': '100%'})
    ]),
    html.Div([
        dcc.Graph(id='annual-energy-chart', style={'width': '100%', 'height': '400px'}),
        dcc.RangeSlider(
            id='year-range-slider',
            min=df['year'].min(),
            max=df['year'].max(),
            value=[df['year'].min(), df['year'].max()],
            marks={str(year): str(year) for year in df['year'].unique()},
            step=None,
            tooltip={"placement": "bottom", "always_visible": True}  # Konfiguracja Tooltip
        )
    ], style={'width': '100%'}),
    
    # Div z informacjami o "Szacowanym zapotrzebowaniu"
html.Div([
        html.H3("Szacowane zapotrzebowanie:"),
            html.Ol([
            html.Li([
                html.H4("Wyznaczenie podstawowych wartości zużycia energii dla pomp ciepła:"),
                html.P("Pompy ciepła są zazwyczaj bardziej efektywne niż tradycyjne systemy ogrzewania, więc ich zużycie energii jest niższe. Przyjmując, że pompa ciepła ma średnią efektywność i biorąc pod uwagę różne rozmiary domów, można założyć następujące wartości:"),
                html.Ul([
                    html.Li("70 m²: Około 50 kWh/m² rocznie"),
                    html.Li("100 m²: Około 50 kWh/m² rocznie"),
                    html.Li("150 m²: Około 50 kWh/m² rocznie")
                ])
            ]),
            html.Li([
                html.H4("Kalkulacja rocznego zapotrzebowania na energię:"),
                html.P("Dla rodziny z dwojgiem dzieci:"),
                html.Ul([
                    html.Li("70 m²: 70m² × 50 kWh/m² = 3500 kWh"),
                    html.Li("100 m²: 100m² × 50 kWh/m² = 5000 kWh"),
                    html.Li("150 m²: 150m² × 50 kWh/m² = 7500 kWh")
                ]),
                html.P("Dla pary (założenie: mniejsze zapotrzebowanie na ciepłą wodę i ogólnie energię):"),
                html.Ul([
                    html.Li("70 m²: 70m² × 50 kWh/m² × 0.85 = 2975 kWh (15% mniej)"),
                    html.Li("100 m²: 100m² × 50 kWh/m² × 0.85 = 4250 kWh"),
                    html.Li("150 m²: 150m² × 50 kWh/m² × 0.85 = 6375 kWh")
                ])
            ]),
            html.Li([
                html.H4("Uwagi:"),
                html.Ul([
                    html.Li("Przyjęto, że zapotrzebowanie na energię jest niższe o 15% dla pary w porównaniu z rodziną z dwojgiem dzieci ze względu na mniejsze zużycie ciepłej wody i mniejszą liczbę urządzeń elektrycznych w użytku."),
                    html.Li("Rzeczywiste zużycie energii może się różnić w zależności od jakości izolacji domu, lokalizacji geograficznej (klimatu), stylu życia mieszkańców oraz wieku i stanu technicznego pompy ciepła.")
                ])
            ])
        ])
    ])
])




# Callback dla dynamicznej aktualizacji i mapy
@app.callback(
    [Output('error-message', 'children'),
     Output('city-input', 'style'),
     Output('annual-energy-chart', 'figure'),
     Output('monthly-weather-chart', 'figure'),
     Output('map-chart', 'figure'),
     Output('monthly-energy-chart', 'figure')],  # Dodanie wyjścia dla nowego wykresu
    [Input('submit-button', 'n_clicks'),
     Input('year-range-slider', 'value')],
    [State('city-input', 'value')]
)
def update_output(n_clicks, year_range, city):
    # Inicjacja pustych wykresów dla przypadku błędu
    empty_fig = {'data': [], 'layout': {'title': {'text': 'Brak danych do wyświetlenia'}}}

    if city not in df['city_name'].unique():
        # Zwraca pustą mapę, wykresy i komunikat o błędzie
        return ('Nie ma takiej miejscowości w bazie', {'border': '2px solid red'}, empty_fig, empty_fig, empty_fig, empty_fig)

    filtered_df = df[(df['city_name'] == city) & (df['year'] >= year_range[0]) & (df['year'] <= year_range[1])]
    annual_energy_fig = px.bar(filtered_df, x='year', y='daily_energy_production_kWh', title=f'Roczna produkcja energii przez lata dla {city}')

    weather_attributes = ['temp','clouds_all', 'humidity', 'wind_speed']
    attribute_labels = ['Temperatura (°C)','Zachmurzenie (%)', 'Wilgotność (%)', 'Prędkość wiatru (m/s)']
    monthly_weather_fig = make_subplots(rows=4, cols=1, subplot_titles=attribute_labels, vertical_spacing=0.2)
    for i, attr in enumerate(weather_attributes, start=1):
        monthly_data = filtered_df.groupby('month')[attr].mean().reset_index()
        monthly_weather_fig.add_trace(go.Scatter(x=monthly_data['month'], y=monthly_data[attr], mode='lines+markers'), row=i, col=1)
        monthly_weather_fig.update_xaxes(title_text="Miesiące", tickvals=list(range(1, 13)), row=i, col=1)

    monthly_weather_fig.update_layout(height=800, showlegend=False, title_text="Średnie miesięczne warunki pogodowe na przestrzeni wszystkich lat", title_x=0.5)
    monthly_weather_fig.update_yaxes(rangemode='tozero')

   # Obliczanie średniej miesięcznej dla wybranego miasta bez ograniczenia lat
    city_data = df[df['city_name'] == city]
    grouped_data = city_data.groupby(['year', 'month'])['daily_energy_production_kWh'].sum().reset_index()
    average_monthly_energy = grouped_data.groupby('month')['daily_energy_production_kWh'].mean().reset_index()

    # Tworzenie wykresu Plotly
    monthly_energy_fig = Figure(data=[
        Bar(x=average_monthly_energy['month'], y=average_monthly_energy['daily_energy_production_kWh'])
    ])
    monthly_energy_fig.update_layout(
        title=f'Średnia miesięczna produkcja energii przez miesiące dla {city} na przestrzeni lat',
        xaxis=dict(title='Miesiąc', tickmode='array', tickvals=list(range(1, 13)), ticktext=['Sty', 'Lut', 'Mar', 'Kwi', 'Maj', 'Cze', 'Lip', 'Sie', 'Wrz', 'Paź', 'Lis', 'Gru']),
        yaxis=dict(title='Średnia miesięczna produkcja energii (kWh)'),
        barmode='group'
    )

        # Znajdź dane dla wybranej lokalizacji
    location_data = df[df['city_name'] == city].drop_duplicates(subset=['city_name'])

    # Zamień "trace 1" na "Stolice" w danych lokalizacji
    location_data['city_name'] = location_data['city_name'].replace('trace 1', 'Stolice')

    # Tworzenie wykresu dla wybranej lokalizacji
    map_fig = px.scatter_geo(location_data, lat='lat', lon='lon', hover_name='city_name',
                            projection='natural earth', title='Lokalizacja na mapie Europy',
                            size_max=20, color_discrete_sequence=["red"])

    # Zamień "trace 1" na "Stolice" w danych stolic
    capitals_df['city_name'] = capitals_df['city_name'].replace('trace 1', 'Stolice')

    # Dodanie punktów dla Stolic z odpowiednią etykietą
    map_fig.add_scattergeo(lat=capitals_df['lat'], lon=capitals_df['lon'], hoverinfo='text',
                        text=capitals_df['city_name'], marker=dict(size=5, color='blue', symbol='circle'), name='Stolice')

    # Dodanie śladu dla wybranej lokalizacji z odpowiednią etykietą
    map_fig.add_scattergeo(lat=location_data['lat'], lon=location_data['lon'], hoverinfo='text',
                        text=city, marker=dict(size=10, color='red', symbol='circle'),
                        name=city)

    # Aktualizacja ustawień mapy
    map_fig.update_geos(
        visible=True, showcountries=True, countrycolor="Black",
        showcoastlines=True, coastlinecolor="Black",
        showland=True, landcolor="LightGreen",
        projection_scale=2
    )
    map_fig.update_layout(
        geo=dict(
            scope='europe', projection_type='orthographic', showframe=False,
            showcoastlines=True, landcolor='rgb(243, 243, 243)',
            countrycolor='rgb(204, 204, 204)',
            lataxis_range=[35, 70], 
            lonaxis_range=[-25, 40], 
            projection_scale=2  
        ),
        legend_title_text='Legenda'
    )

    return '', {'border': '1px solid green'}, annual_energy_fig, monthly_weather_fig, map_fig, monthly_energy_fig


if __name__ == '__main__':
    app.run_server(debug=True, port=8050)  # 


### ----------------------------------------------------------------------------------------------------

### PROBLEM Z WYKRESEM ZA MIESIAC (NOTATKA)

- daje rozne wartosci? w kod2 jest poprawne bo srednia produkcja energi na przestrzeni lat jest okolo 300kWh w kazdym miesiacu.

- w KOD1 jest to na poziomie 10kWh? co jest nie tak?

- Analizując oba kody, można zauważyć kluczowe różnice, które mogą tłumaczyć, dlaczego wartości średniej miesięcznej produkcji energii są różne.

#### Zakres danych:

- Kod 1 skupia się na jednym wybranym mieście, używając filtru df['city_name'] == city. To ogranicza dane tylko do tego miasta, co może mieć wpływ na różnice w wynikach.
- Kod 2 oblicza średnie dla wszystkich miast w zbiorze danych (df_weather_db), a następnie grupuje dane po miesiącach, latach i nazwach miast.
#### Metoda grupowania danych:

- Kod 1 grupuje dane tylko według miesiąca i oblicza średnią dla każdego miesiąca. W tym przypadku, brak jest agregacji rocznej, co może powodować, że średnia jest obliczana na mniejszej liczbie danych, szczególnie jeśli dane dla poszczególnych lat są nieregularne lub mają odstające wartości.
- Kod 2 najpierw sumuje dane miesięczne dla każdego roku i lokalizacji, a potem oblicza średnią dla tych sum, co prowadzi do bardziej znormalizowanych i uśrednionych wyników przez różne lata.
#### Analiza wartości ekstremalnych i rozkład danych:

- W Kodzie 1, brak jest jakiejkolwiek normalizacji lub ważenia danych rocznych, co może skutkować większą wrażliwością na wartości ekstremalne lub anomalie w danych dla wybranego miasta.
- Kod 2, przez agregowanie i uśrednianie danych przez lata i miasta, może lepiej radzić sobie z anomalią danych, co skutkuje bardziej stabilnymi i reprezentatywnymi wynikami.

### Szacowane zapotrzebowanie:
1. Wyznaczenie podstawowych wartości zużycia energii dla pomp ciepła:
#### Pompy ciepła są zazwyczaj bardziej efektywne niż tradycyjne systemy ogrzewania, więc ich zużycie energii jest niższe. Przyjmując, że pompa ciepła ma średnią efektywność i biorąc pod uwagę różne rozmiary domów, można założyć następujące wartości:
- 70 m²: Około 50 kWh/m² rocznie
- 100 m²: Około 50 kWh/m² rocznie
- 150 m²: Około 50 kWh/m² rocznie
2. Kalkulacja rocznego zapotrzebowania na energię:
#### Dla rodziny z dwojgiem dzieci:
- 70 m²: 70m2×50kWh/m2=3500kWh70m2×50kWh/m2=3500kWh
- 100 m²: 100m2×50kWh/m2=5000kWh100m2×50kWh/m2=5000kWh
- 150 m²: 150m2×50kWh/m2=7500kWh150m2×50kWh/m2=7500kWh
#### Dla pary (założenie: mniejsze zapotrzebowanie na ciepłą wodę i ogólnie energię):
- 70 m²: 70m2×50kWh/m2×0.85=2975kWh70m2×50kWh/m2×0.85=2975kWh(15% mniej)
- 100 m²: 100m2×50kWh/m2×0.85=4250kWh100m2×50kWh/m2×0.85=4250kWh
- 150 m²: 150m2×50kWh/m2×0.85=6375kWh150m2×50kWh/m2×0.85=6375kWh
3. Uwagi:
 - Przyjęto, że zapotrzebowanie na energię jest niższe o 15% dla pary w porównaniu z rodziną z dwojgiem dzieci ze względu na mniejsze zużycie ciepłej wody i mniejszą liczbę urządzeń elektrycznych w użytku.
- Rzeczywiste zużycie energii może się różnić w zależności od jakości izolacji domu, lokalizacji geograficznej (klimatu), stylu życia mieszkańców oraz wieku i stanu technicznego pompy ciepła.
#### Wnioski:
- Te przybliżone obliczenia dają ogólny obraz potencjalnego zapotrzebowania na energię w domach ogrzewanych za pomocą pompy ciepła. Warto jednak zauważyć, że indywidualne warunki mogą znacząco wpłynąć na rzeczywiste zużycie energii. Dla dokładniejszej analizy i optymalizacji zużycia energii zalecane jest przeprowadzenie audytu energetycznego domu.

Aus <https://chatgpt.com/c/f54165fc-a2e6-4591-987f-8b4ee7be6303> 
